In [14]:
import re
import requests
import json
import sqlite3
import glob
import urllib.parse
from bs4 import BeautifulSoup
import MeCab as mc
from collections import Counter
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
conn = None

#  データベースに接続する
def connect():
    global conn
    conn = sqlite3.connect('./weblogs.db')

def close():
    conn.close()
    
def create_table():
    conn.execute('DROP TABLE IF EXISTS weblogs')
    conn.execute("""CREATE TABLE weblogs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title text,
            content text
            )""")

#  Countriesテーブルにレコードを挿入
def load(title, content):
    conn.execute(
        'INSERT INTO weblogs (title, content) VALUES (?, ?)', (title, content))
    conn.commit()

def get_content(name):
    return [record[0] for record in 
               conn.execute(
            'SELECT content FROM countries WHERE title = ?',
            (name))]

def get_all_contents(limit, offset=0):
    return [record[0] for record in 
               conn.execute(
            'SELECT content FROM countries LIMIT ? OFFSET ?',
            (limit, offset))]

In [12]:
url = "https://www3.ttc-net.co.jp/member-page/wp-login.php?"

login_info  = {
    "log": "y-watanabe",
    "pwd": "Vocaula9101"
}


session = requests.session()
r = session.post(url, data=login_info)
bs = BeautifulSoup(r.text, "html.parser")

In [21]:
connect()
create_table()
close()

In [22]:
a = bs.find("div", class_="menu-header")
a = a.find_all("a")
link = a[1].get("href")

res = session.get(link)
soup = BeautifulSoup(res.text, "html.parser")
link = insert_weblogs(soup)
while link:
    res = session.get(link)
    soup = BeautifulSoup(res.text, "html.parser")
    link = insert_weblogs(soup)


□ttc1441　小学4年が基本情報技術者試験合格
□ttc1440　新人向け特別講義：PDC(A)のP
□ttc1439　マナー：ジャニーズ編
□ttc1438　KBM新宿藩にて西岡君プレゼン
□ttc1437　ttcは「論語会社」か「算盤会社」か
□ttc1436　zoom　ブレイクアウトルーム
□ttc1435　ＰＭ研修として試写「愛と青春の旅立ち」
□ttc1434　メルカリで御朱印！
□ttc1433　日本一の下足番になれ！
□ttc1432　「一所懸命」vs「一生懸命」おやっ！
□ttc1431　Thunderbird（メーラー）が使いにくい！
□ttc1430　メーラーのアドレス帳・・大丈夫！
□ttc1429　情報学　ジャーナリスト山村氏来社
□ttc1428　行動目標面接前の★必須事項★
□ttc1427　ファルダ整理・・どうしてる？
□ttc1426　doreming高崎氏 Wedgeに載る！
□ttc1425　tBCサービス化の為に！　アンケート実施
□ttc1424　グループウェア　何にする？
□ttc1423　tBC配布しよっ！
□ttc1422　とあるアポのDM
□ttc1421　行動目標を立てる(5/13WBA）→姿勢から♪
□ttc1420　行動目標を立てる(5/13WBA）
□ttc1419　行動目標を振り返る
□ttc1418　オリンピックチケット予約開始
□ttc1417　日本の祭り
□ttc1416　至世會（in京都）に参加してきた
□ttc1415　Pocetalk が見つかった！
□ttc1414　連休・・何してた！？
□ttc1413　教育フェスタ　課題１の裏話
□ttc1412　タクシーで一万円札
□ttc1411　10連休も・・（サザエさん症候群？）
□ttc1410  一般参賀に行こう
□ttc1409　令和を迎え思ふこと(ttc通信）
□ttc1408　「一生懸命塾」訪問
□ttc1407　令和元年5月1日　ハナミズキを植樹
□ttc1406　平成最終日の皇居
□ttc1405　平成を振り返る
□ttc1404　平成最後の日（4/30火）
□ttc1403　三種の神器
□ttc1402　10連休をどう活かすか！
□ttc1400　「夢」 と 「志」
□ttc1401　朝会でTBCをプレゼン
□ttc1399　地元

AttributeError: 'NoneType' object has no attribute 'get'

In [11]:
# HTMLを受け取りブログの内容だけを取り出す
def get_text(html):
    soup = BeautifulSoup(html.text, "html.parser")
    return soup.find("p").text

In [20]:
# スープされたHTMLを受け取りブログのリンク先のテキストをDBに格納する
def insert_weblogs(soup):
    connect()
    for h2 in soup.find_all("h2", class_="entry-title"):
        a = h2.find("a")
        if a:
            title = a.text
            link  = a.get("href")
            text  = get_text(session.get(link))
            load(title, text)
            print(title)
    close()
    return soup.find("a", class_="nextpostslink").get("href")